# Web Scraping METROCUADRADO

## Librerías

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

## Requerimiento a la pagína

## Selección de grilla

In [80]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')

driver.quit()

In [81]:
house_grill = soup.find('ul', class_ = 'browse-results-list')

In [82]:
df = pd.DataFrame()

## Nombre Publicación

In [83]:
name_houses_list = []
price_houses = []
area_houses = []
rooms_houses = []
baths_houses = []

for name_houses in house_grill:

    name = name_houses.find('h2', class_ = 'card-title').text
    name_houses_list.append(name)

    info = name_houses.find_all('p', class_ = 'card-subitem')
    
    price_houses.append(info[0].text)
    area_houses.append(info[1].text)
    rooms_houses.append(info[2].text)
    baths_houses.append(info[3].text)


df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
df['Precio_inmueble'] = price_houses
df['Area_construida'] = area_houses
df['Habitaciones'] = rooms_houses
df['Banios'] = baths_houses
print(df.head(5))

                       Nombre_publicacion Precio_inmueble Area_construida  \
0  Casa en Venta, colinas de suba, Bogot…  $2.600.000.000          290 m²   
1          Casa en Venta, Canelón, Cajicá  $1.250.000.000          274 m²   
2         Casa en Venta, CHUNUGUA, Cajicá  $2.200.000.000          312 m²   
3    Apartamento en Venta, La Colina Cam…  $1.050.000.000       158.32 m²   
4   Casa en Venta, Vereda Cerca de Piedr…  $1.850.000.000          455 m²   

  Habitaciones Banios  
0            3      4  
1            5      5  
2            3      5  
3            3      5  
4            4      5  


## Información publicación

In [84]:
df

,Nombre_publicacion,Precio_inmueble,Area_construida,Habitaciones,Banios
0,"Casa en Venta, colinas de suba, Bogot…",$2.600.000.000,290 m²,3,4
1,"Casa en Venta, Canelón, Cajicá",$1.250.000.000,274 m²,5,5
2,"Casa en Venta, CHUNUGUA, Cajicá",$2.200.000.000,312 m²,3,5
3,"Apartamento en Venta, La Colina Cam…",$1.050.000.000,158.32 m²,3,5
4,"Casa en Venta, Vereda Cerca de Piedr…",$1.850.000.000,455 m²,4,5
5,"Casa en Venta, COLINA DE LOS VIENT…",$8.000.000.000,736 m²,5,5
6,"Apartamento en Venta, LAGOS DE CO…",$1.190.000.000,172 m²,3,5
7,"Casa en Venta, SAN JOSE DE BAVARIA,…",$2.680.000.000,323 m²,4,5
8,"Casa en Venta, COLINAS DE SUBA, Bo…",$1.700.000.000,380 m²,3,5
9,"Casa en Arriendo, LISBOA, Bogotá …",$10.000.000,198 m²,4,4


## Multiples páginas

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [84]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')


df_final = pd.DataFrame()


try:
    # Espera hasta que la página cargue completamente
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "browse-results-list")))

    # Ciclo para navegar a través de las páginas
    current_page = 1
    while True:
        
        # Extrae datos aquí, por ejemplo, nombres de productos
        
        df = pd.DataFrame()
        
        #elements = driver.find_elements(By.CLASS_NAME, "browse-results-list")
        house_grill = soup.find('ul', class_ = 'browse-results-list')
        name_houses_list = []
        price_houses = []
        area_houses = []
        rooms_houses = []
        baths_houses = []

        for name_houses in house_grill:

            name = name_houses.find('h2', class_ = 'card-title').text
            name_houses_list.append(name)

            info = name_houses.find_all('p', class_ = 'card-subitem')
            
            price_houses.append(info[0].text)
            area_houses.append(info[1].text)
            rooms_houses.append(info[2].text)
            baths_houses.append(info[3].text)


        df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
        df['Precio_inmueble'] = price_houses
        df['Area_construida'] = area_houses
        df['Habitaciones'] = rooms_houses
        df['Banios'] = baths_houses
        df['Pagina'] = current_page
        #print(df.head(5))
        print("Página:",current_page)

        df_final = pd.concat([df_final, df], ignore_index=True)

        # Intenta encontrar el número de la siguiente página y hacer clic en él
        next_page = current_page + 1
        page_elements = driver.find_elements(By.CLASS_NAME, "pagination")

        find_element = driver.find_element(By.CLASS_NAME, "pagination")

        # Ejecutar JavaScript para hacer scroll hasta el elemento
        driver.execute_script("arguments[0].scrollIntoView(true);", find_element)        

        found_next_page = False 

        time.sleep(10)       

        # # Iterar sobre cada elemento para imprimir su texto
        for page_element in page_elements:
            for page in page_element.find_elements(By.CLASS_NAME, 'page-link'):               
                if page.tag_name == 'a':
                    if page.text == str(next_page):                   
                        print(page)
                        page.click()
                        found_next_page = True
                        current_page = next_page
                        time.sleep(5)  # Espera para asegurar que la página se haya cargado completamente
                        break
        
        if not found_next_page:
            print("Reached last page.")
            break
# except Exception as e:
#     print(f"An error occurred: {e}")
finally:
    # Cierra el navegador
    driver.quit()
    


Página: 1
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.144")>
Página: 2
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.145")>
Página: 3
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.146")>
Página: 4
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.147")>
Página: 5


KeyboardInterrupt: 

In [91]:
df_final.duplicated().value_counts()

False    245
True       5
Name: count, dtype: int64